In [3]:
import numpy as np
from os import listdir
from os.path import isfile, join
import mesa_reader as m_r
import pandas as pd

# Functions:

def tau_lim_finder(all_zones, num_zones, tau_column, zone_stop):
    
    for i in range(num_zones):
        
        if abs(zone_stop - all_zones[i][tau_column]) < abs(zone_stop - all_zones[i-1][tau_column]):
            #finds which tau value is closest to 1
            
            tau_lim = i #tau_lim is the row at which tau is closest to 1
            
    return(tau_lim);

def total_mass_finder(all_zones, mass_column, tau_lim):
    
    total_mass = 0
    
    for i in range(tau_lim + 1):
        
        total_mass += all_zones[i][mass_column] 
        #goes through and adds all masses up to tau_lim
        
    return(total_mass);

def total_mass_frac_finder(all_zones, mass_column, tau_lim, tau_column, total_mass, nuclides_begin):
    
    total_mass_fracs = [] 
    
    for i in range(nuclides_begin, tau_column): #starts at neutrons and stops before tau column
        
        nuclide_total = 0
        
        for j in range(tau_lim + 1): #this iterates over desired zones
            
            nuclide_total += all_zones[j][i]*all_zones[j][mass_column]
            # goes through and adds up all of the mass fractions for a single nuclide in all desired zones
            # each mass fraction for given nuclide in a zone is weighted by multiplying it by zone mass 
            # before being added to total mass fraction
            
        total_mass_fracs.append(nuclide_total / total_mass) 
        #once all weighted mass fractions have been added, we divide by combined mass of all zones to 
        #get the weighted average mass fraction for each nuclide
        
        
    return(total_mass_fracs); 



def percent_diff_finder(profile_mass_fracs, num_runs, base_profile):
    
    percent_diff = [[0 for x in range(len(total_mass_fracs))] for y in range((num_runs//2)-1)] 
    #creates a 2d array to fill in percent difference between each nuclide and that of the base profile,
    #for every profile

    for i in range(0, (num_runs//2)-1):
        # num_runs is set as 200, but it only takes odd numbers up to 199, so there is half the number
    
        for k in range(len(total_mass_fracs)):
        
            if profile_mass_fracs[base_profile][k] != 0:
            #this deals with dividing by zero mass fraction values
            
                percent_diff[i][k]= abs((profile_mass_fracs[base_profile][k] - profile_mass_fracs[i+1][k])/profile_mass_fracs[base_profile][k])
                # Equation: |(x_ibase - x_iprofile)/ x_ibase|
            
            else:
            #right now, if the base profile mass fraction is 0, I set the comparative mass fraction to zero
        
                percent_diff[i][k] = 0
            
    #percent_diff = percent_diff[:-1:]
    #takes off last array element, because it is comparing the base profile to itself
            
    return(percent_diff)

# Adjustable Variables Below

num_zones = 790 #minus 1 because 0th index
mass_column = 1 #1st row in spreadsheet as 0th row as index
#tau_column =  #for when tau is approximately 1
zone_stop = 1 # tau value that we want to stop counting rows at
nuclides_begin = 9 #column that the actual nuclide information begins
num_profiles = 1 #including 0th! So 6 profiles means variable is 5
base_profile = 0 #the position of the base profile that you're comparing the other files too, in the list below
num_runs = 200

profile_mass_fracs = []

for j in range(-1, num_runs, 2):
    print(j)
    
    if j == -1:
        path = r'/Users/beecontreras/Desktop/2019_Duke_TUNL_REU/phy-champagne-lauer/baseline/LOGS/'
    else: 
        s = str(j)

        path = r'/Users/beecontreras/Desktop/2019_Duke_TUNL_REU/runs_x100_1/'+ s + '/LOGS/'
            #insert the path to the folder containing the stellar profiles/ leave "r'" if you have a mac.

        file = 'profile1.data'
        new_path = path + file
    #creates path for which specific profile we wish to open

    with open(new_path,'r') as f:
    #Note: leave "r'" in front of file path, even when changing the path

            zones_begin = 0   #counter so that only lines with zone information get turned into float arrays

            all_zones = []    #list that zone_elements array is fed into, so every index is info. for an entire zone

            nuclide_label = []

            for line in f:

                zones_begin += 1

                zone_info = line.split(" ")    #splits the individual numbers in the lines

                zone_info = np.array(list(filter(None, zone_info)))[:-1:] 
                    #gets rid of spaces and '\n' and puts each list element into the array

                    
                if zones_begin == 5:
                    tau_column = line.split(" ")
                    #print(tau_column)
                    tau_column = np.array(list(filter(None, tau_column)))[-2]
                    #print(tau_column)
                    tau_column = tau_column.astype(int) -1
                if zones_begin == 6:

                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
                # code related to this if statement creates nuclide dictionary                                    #
                # connecting each nuclear species to it's respective element position within the total_mass_frac  #
                # array. To search for specific element, insert "nuclides['name of nuclide']" into the second     #
                # index of total_mass_fracs[i][j] (i.e. the j index)                                              #
                # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

                    nuclides = {}

                    for i in range(tau_column - nuclides_begin):

                        nuclides[zone_info[i + nuclides_begin]] = i

                if zones_begin >= 7 : 
                    #distinguishes zone information from other info and labels, like the names of the nuclides 

                    zone_elements = zone_info.astype(float)

                    all_zones.append(zone_elements) #puts each zone_element iteration into all_zones list

            num_zones = len(all_zones)
                #sets the number of zones to that of the number of zone rows in this particular file iteration

            all_zones = np.array(all_zones)
                #format: all_zones[i][j]
                #first bracket is the zone you want -minus one because zeroth index-
                #second bracket is what component you want from a single zone, i.e. mass fraction / mass of zone / zone number

        #functions defined at bottom of script
    tau_lim = tau_lim_finder(all_zones, num_zones, tau_column, zone_stop)

    total_mass = total_mass_finder(all_zones, mass_column, tau_lim)

    total_mass_fracs = total_mass_frac_finder(all_zones, mass_column, tau_lim, tau_column, total_mass, nuclides_begin)

    profile_mass_fracs.append(total_mass_fracs)
    #puts each profile mass fractions into an array of them

#print(profile_mass_fracs)
percent_diff = percent_diff_finder(profile_mass_fracs, num_runs, base_profile)

for i in range((num_runs//2)):
    print(percent_diff[i],'\n')
        #prints the percent difference for every nuclide in each profile compared to the base profile for all profiles

-1
1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199


IndexError: list index out of range

In [23]:

def percent_diff_finder(profile_mass_fracs, num_runs, base_profile):
    
    percent_diff = [[0 for x in range(len(total_mass_fracs))] for y in range(len(profile_mass_fracs))] 
    #creates a 2d array to fill in percent difference between each nuclide and that of the base profile,
    #for every profile

    for i in range(0, len(profile_mass_fracs)):
        print(i)
        print(len(total_mass_fracs))
        #print(len(profile_mass_fracs))
        #print((num_runs//2))
        # num_runs is set as 200, but it only takes odd numbers up to 199, so there is half the number
    
        for k in range(len(total_mass_fracs)):
        
            if profile_mass_fracs[base_profile][k] != 0:
            #this deals with dividing by zero mass fraction values
            
                percent_diff[i][k]= abs((profile_mass_fracs[base_profile][k] - profile_mass_fracs[i][k])/profile_mass_fracs[base_profile][k])
                # Equation: |(x_ibase - x_iprofile)/ x_ibase|
            
            else:
            #right now, if the base profile mass fraction is 0, I set the comparative mass fraction to zero
        
                percent_diff[i][k] = 0
            
    #percent_diff = percent_diff[:-1:]
    #takes off last array element, because it is comparing the base profile to itself
            
    return(percent_diff)
percent_diff = percent_diff_finder(profile_mass_fracs, num_runs, base_profile)

for i in range(len(profile_mass_fracs)):
    print(percent_diff[i],'\n')
        #prints the percent difference for every nuclide in each profile compared to the base profile for all profiles

0
305
1
305
2
305
3
305
4
305
5
305
6
305
7
305
8
305
9
305
10
305
11
305
12
305
13
305
14
305
15
305
16
305
17
305
18
305
19
305
20
305
21
305
22
305
23
305
24
305
25
305
26
305
27
305
28
305
29
305
30
305
31
305
32
305
33
305
34
305
35
305


IndexError: list index out of range

In [ ]:
with open(r'/Users/beecontreras/Desktop/2019_Duke_TUNL_REU/runs_x100_1/reaction_list_305.txt','r') as f:
    # opens the reaction list that corresponds to each file/profile looked at

    for line in f:
        
        reaction_list = f.readlines()
        # reads in each line
        
    reaction_list = reaction_list[0:99]
    # only takes the first 99 reactions, because we only looked at 99 profiles (not including the base profile)

In [ ]:
df = pd.DataFrame(data = np.transpose(percent_diff),index = nuclides, columns = reaction_list)
# creates a data frame using pandas - essentially tells code below where the data is stored 

writer = pd.ExcelWriter('sensitivity.xlsx', engine='xlsxwriter')
df.to_excel(writer, index=True)
writer.save()
# The above creates, writes in the data, then saves and excel spreadsheet